# Rendu intermédiaire: récupération d'une grande masse de données

Choix de hubs subjectifs :  Le Figaro, qui est de "droite gaulliste, libérale et conservatrice".  Libération, qui est "un journal libertaire de gauche"

1. Initialisation de Tweepy

In [5]:
import tweepy

In [6]:
from tweepy import OAuthHandler

In [7]:
consumer_key = 'L2mPv8Oxjs2mLtBEWZZiyDy0W'
consumer_secret = 'idPHQogjNLSKdo7ns8exk9ZXSbIsqcd4UsXd1vofliwj1s2iZj'
access_token = '1413867470-bWJRTSz2Ece90tG1Z5IU9pmPZQvdWR10xrWqNBL'
access_secret = '6Z0X9eJ0dLlYJj8JrqX6VcNMOOsTWpaxaFosJhiv5Xkoa'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

2. Initialisation des hubs

In [8]:
hub1 = api.get_user('Le_Figaro')
hub2 = api.get_user('libe')
print(hub1.screen_name)
print(hub1.followers_count)
print('-----------')
print(hub2.screen_name)
print(hub2.followers_count)

Le_Figaro
3339476
-----------
libe
3180964


3. Récupération des comptes qui suivent les hubs 

In [9]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

In [12]:
followerids_hub1 =[]
for user in tweepy.Cursor(api.followers_ids, screen_name=hub1.screen_name,count=5000).items():
    followerids_hub1.append(user)
    if len(followerids_hub1) > 300000: #10%
        break
print(len(followerids_hub1))

Rate limit reached. Sleeping for: 876
Rate limit reached. Sleeping for: 890
Rate limit reached. Sleeping for: 888
Rate limit reached. Sleeping for: 889


300001


In [13]:
followerids_hub2 =[]
for user in tweepy.Cursor(api.followers_ids, screen_name=hub2.screen_name,count=5000).items():
    followerids_hub2.append(user)
    if len(followerids_hub2) > 300000: #10%
        break
print(len(followerids_hub2))

Rate limit reached. Sleeping for: 804
Rate limit reached. Sleeping for: 889
Rate limit reached. Sleeping for: 886
Rate limit reached. Sleeping for: 889


300001


In [14]:
import pandas as pd 
import numpy as np

In [15]:
df1 = pd.DataFrame(followerids_hub1)
df1.to_csv('followersid_hub1.csv', sep ='\t') 

In [16]:
df2 = pd.DataFrame(followerids_hub2)
df2.to_csv('followersid_hub2.csv', sep ='\t') 

In [17]:
users_hub1 = []
for i in range(int(len(followerids_hub1)/100) + 1):            
    fin_cycle = min((i + 1) * 100, len(followerids_hub1))
    users_hub1.extend(
        api.lookup_users(user_ids=followerids_hub1[i * 100:fin_cycle])                
    )

In [18]:
df3 = pd.DataFrame(users_hub1)
df3.to_csv('users_hub1.csv', sep ='\t')

In [19]:
users_hub2 = []
for i in range(int(len(followerids_hub2)/100) + 1):            
    fin_cycle = min((i + 1) * 100, len(followerids_hub2))
    users_hub2.extend(
        api.lookup_users(user_ids=followerids_hub2[i * 100:fin_cycle])                
    )

In [20]:
df4 = pd.DataFrame(users_hub2)
df4.to_csv('users_hub2.csv', sep ='\t')

4. Récupération des abonnements des followers des hubs

# A partir d'ici c'est du test à tout va

In [21]:
import twint

In [73]:
usernames_nbfollowers_figaro = {}
usernames_nbfollowers_libe = {}
for i in range(len(users_hub1)):
    usernames_nbfollowers_figaro[users_hub1[i].screen_name] = users_hub1[i].followers_count

for j in range(len(users_hub2)):
    usernames_nbfollowers_libe[users_hub2[j].screen_name] = users_hub2[j].followers_count
    

ValueError: If using all scalar values, you must pass an index

In [79]:
df5 = pd.DataFrame.from_dict(usernames_nbfollowers_figaro, orient ='index')
df5.to_csv('usernames_nbfollowers_figaro.csv', sep =',')
df6 = pd.DataFrame.from_dict(usernames_nbfollowers_libe, orient ='index')
df6.to_csv('usernames_nbfollowers_libe.csv', sep =',')

In [66]:
import time
for i in range(len(usernames_hub1)):
    c = twint.Config()
    c.Username = usernames_hub1[i]
    c.Store_csv = True
    c.Output = "all_hub1/" + usernames_hub1[i] + '.csv'
    try:
        twint.run.Following(c)
    except:
        time.sleep(2)

BalmePatricia
LeaderNewsCI
AlexFre37328341
PATTYCIA1957
Europe1
sofoot
KingOfZeDay
lachainelequipe
LMDFoot_
lequipedusoir
gdufy
ggaretier
CanalplusSport
DenisEstelle
RaymondDomenech
hpenot_lequipe
suspect_95
footmercato
PassionFootClub
Footballogue


KeyboardInterrupt: 

CRITICAL:root:twint.get:User:
CRITICAL:root:twint.get:User:
CRITICAL:root:twint.get:User:


In [45]:
users_hub1[5].friends_count

196

In [68]:
for i in range(len(usernames_hub2)):
    c = twint.Config()
    c.Username = usernames_hub2[i]
    c.Store_csv = True
    c.Output = "all_hub2/" + usernames_hub2[i] + '.csv'
    try:
        twint.run.Following(c)
    except:
        time.sleep(2)

KeyboardInterrupt: 

In [69]:
abonnementsids_hub1 = {}
for i in range(len(users_hub1)):
    # Pour chaque utilisateur, on récupère ses abonnements avec la même méthode
    tmpids = []
    # Ne pas oublier la gestion d'erreur pour les comptes privés (jusque 13%), il y en a beaucoup !!
    try:
        for abonnement in tweepy.Cursor(api.friends_ids, screen_name=users_hub1[i].screen_name, count=5000).items():
            tmpids.append(abonnement)
    except tweepy.TweepError: 
        print("Erreur utilisateur, au suivant")
    abonnementsids_hub1[users_hub1[i].screen_name] = tmpids
len(abonnementsids_hub1)


Erreur utilisateur, au suivant


Erreur utilisateur, au suivant
Erreur utilisateur, au suivant


Erreur utilisateur, au suivant
Erreur utilisateur, au suivant
Erreur utilisateur, au suivant
Erreur utilisateur, au suivant


Erreur utilisateur, au suivant
Erreur utilisateur, au suivant


Erreur utilisateur, au suivant


Erreur utilisateur, au suivant
Erreur utilisateur, au suivant
Erreur utilisateur, au suivant
Erreur utilisateur, au suivant


Erreur utilisateur, au suivant
Erreur utilisateur, au suivant
Erreur utilisateur, au suivant
Erreur utilisateur, au suivant


Erreur utilisateur, au suivant


Erreur utilisateur, au suivant


KeyboardInterrupt: 

In [71]:
len(abonnementsids_hub1)

144